# Predator Prey Model

The predator-prey model consists of a pair of first-order nonlinear differential equations, commonly used to describe the dynamics of biological systems in which two species interact, with one acting as a predator and the other as prey.

We use this model to look into the behaviour of different integrators available in ReMKiT1D.

### Setting up the Context and Simulation Grid

In [ ]:
# import libraries and functions
import numpy as np

import RMK_support as rmk
from RMK_support import DiagonalStencil

In [ ]:
# initialise context
rk = rmk.RMKContext()

# IO and MPI context setting
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_pred_prey/")
rk.mpiContext = rmk.MPIContext(numProcsX=1)

In [ ]:
# initialise grid (defaults to normalised units)

rk.grid = rmk.Grid(np.ones(1), interpretXGridAsWidths=True)

### The equations

The predator-prey system can be written as

$$ \frac{dx}{dt} = (\alpha - \beta y)x $$
$$ \frac{dy}{dt} = (\delta x - \gamma)y, $$

where $\alpha$ and $\beta$ are the prey growth and death factors, and $\delta$ and $\gamma$ are the predator growth and death factors. $x$ and $y$ are prey and predator numbers in arbitrary units.

### Variables

The only variables needed are the implicit predator and pray variables, $x$ and $y$. Additionally, we can set the four constants $\alpha$, $\beta$, $\gamma$, and $\delta$. 

**NOTE**: The `x` name is reserved for the grid, an thus cannot be used for variables.

In [ ]:
# define constants
alpha = 1.1
beta = 0.4
gamma = 0.4
delta = 0.1

# predator and prey variables

xInit = 10*np.ones(len(rk.grid.xGrid))
yInit = 2 *np.ones(len(rk.grid.xGrid))

x1 = rmk.Variable("x1", rk.grid, data=xInit)
y1 = rmk.Variable("y1", rk.grid, data=yInit)

x2 = rmk.Variable("x2", rk.grid, data=xInit)
y2 = rmk.Variable("y2", rk.grid, data=yInit)

# add variables to context
rk.variables.add(x1, y1, x2, y2)

### Models

Using the diagonal stencil, the predator-prey equations can be defined and added to our model.

In [ ]:
# initialise models
model_BE = rmk.Model(name="predator_prey_BE")
model_CV = rmk.Model(name="predator_prey_CV")

# define model equations

diag = DiagonalStencil()

model_BE.ddt[x1] += alpha*diag(x1).rename("term1_1").regroup(implicitGroups=[1]) - beta*diag(y1*x1).rename("term2_1").regroup(implicitGroups=[1])
model_BE.ddt[y1] += -gamma* diag(y1).rename("term3_1").regroup(implicitGroups=[2]) + delta*diag(x1*y1).rename("term4_1").regroup(implicitGroups=[2])

model_CV.ddt[x2] += alpha*diag(x2).rename("term1_2").regroup(implicitGroups=[3]) - beta*diag(y2*x2).rename("term2_2").regroup(implicitGroups=[3])
model_CV.ddt[y2] += -gamma* diag(y2).rename("term3_2").regroup(implicitGroups=[4]) + delta*diag(x2*y2).rename("term4_2").regroup(implicitGroups=[4])

# add models to context
rk.models.add(model_BE, model_CV)

### Integration Scheme

Finally, the integration scheme is set up. The predator-prey model is a good place to showcase the different behaviour of time integrators available in ReMKiT1D. The Backward Euler integrator is first order, while the methods used in CVODE are variable-order.

To compare multiple integrators, we split integration so that the different models are evolved using different integrators. Each integrator is applied for the full duration of the time step and with time evolution disabled for all but the leftmost integrator (applied last). This ensures that no steps are skipped in any of the integrators.

In [ ]:
# set integrators for comparison of integration methods
integrator1 = rmk.BDEIntegrator(name="BDE", nonlinTol=1e-12, absTol=10.0, convergenceVars=[x1, y1])
integrator2 = rmk.CVODEIntegrator(name="CVODE")

# set integration steps
integrationStep1 = rmk.IntegrationStep(name="BE", integrator=integrator1)
integrationStep2 = rmk.IntegrationStep(name="CV", integrator=integrator2)

# add all models in context to integration steps
integrationStep1.add(model_BE)
integrationStep2.add(model_CV)

# define integration scheme 
# because we want to have integrationStep1 and integrationStep2 to perform integrations in parallel we 
# disable time evolution on the first step applied (the rightmost step)
rk.integrationScheme = rmk.IntegrationScheme(dt=0.1, steps=integrationStep2(1.0)*integrationStep1(1.0).disableTimeEvo())
rk.integrationScheme.setOutputPoints(outputPoints=[0.1*i for i in range(1,500)])


### Generate PDF

In [ ]:
rk.generatePDF("Predator-Prey Model")

### Create config file

In [ ]:
rk.writeConfigFile()

# Data Analysis

Load data from HDF5 files

In [ ]:
loadedData = rk.loadSimulation()
dataset = loadedData.dataset
dataset

Set plotting parameters

In [ ]:
# load plotting extensions
import panel as pn
import holoviews as hv
import matplotlib.pyplot as plt

import RMK_support.dashboard_support as ds


hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)
hv.output(max_frames=1000)


Plot evolution of variables over time

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(dataset,rk.grid)

dashboard.fluidMultiComparison(["x1","y1", "x2", "y2"],fixedPosition=True)


### Comparison of Integrators

As shown below, the CVODE method is less dissipative compared to the implicit Backward Euler integrator. The inward spiral observed with Backward Euler indicates that the populations artificially decay over time, likely due to numerical dissipation or integration errors. In contrast, the CVODE method better preserves the system's dynamics, leading to less energy loss.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2)

# Plot Backward Euler Method
axes[0].plot(dataset["x1"], dataset["y1"], color="blue")
axes[0].set_title("Backward Euler")
axes[0].set_xlabel("x")
axes[0].set_ylabel("y")

# Plot CVODE Method
axes[1].plot(dataset["x2"], dataset["y2"], color="red")
axes[1].set_title("CVODE")
axes[1].set_xlabel("x")
axes[1].set_ylabel("y")

plt.tight_layout()